In [0]:
%python
# from pyspark.sql.types import StructType
# from pyspark.sql.functions import col, row_number
# from pyspark.sql.window import Window
# from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType
# from pyspark.sql.functions import concat, regexp_extract, date_format, current_timestamp
# from pyspark.sql.functions import when, col, to_timestamp, to_date, regexp_replace
# from pyspark.sql.functions import sha2, concat_ws, coalesce, lit, trim, struct
# from pyspark.sql.functions import to_json, expr, to_utc_timestamp
# from pyspark.sql.types import StructType, ArrayType
# from pyspark.sql.functions import transform
# import re
# from functions import create_table_if_not_exists, get_function, silver_scd2_transform
# from functions import read_snapshot_windowed, silver_standard_transform, overwrite_table
# from pyspark.sql.functions import col, row_number
# from pyspark.sql.window import Window

# settings = {
#     "read_function": "functions.stream_read_table",
#     "transform_function": "functions.silver_standard_transform",
#     "write_function": "functions.write.stream_upsert_table",
#     "upsert_function": "functions.upsert_microbatch",
#     "src_table_name": "edsm.bronze.systemsWithCoordinates",
#     "dst_table_name": "edsm.silver.systemsWithCoordinates",
#     "build_history": "true",
#     "business_key": [
#         "id"
#     ],
#     "surrogate_key": [
#         "name",
#         "coords.x",
#         "coords.y",
#         "coords.z",
#         "date"
#     ],
#     "data_type_map": {
#         "date": "timestamp"
#     },
#     "readStreamOptions": {},
#     "writeStreamOptions": {
#         "mergeSchema": "false",
#         "checkpointLocation": "/Volumes/edsm/silver/utility/systemsWithCoordinates7days/_checkpoints/",
#         "delta.columnMapping.mode": "name"
#     },
#     "ingest_time_column": "derived_ingest_time"
# }


# df = read_snapshot_windowed(spark, settings)
# df = silver_standard_transform(df, settings, spark)

# overwrite_table(df, settings, spark)

In [0]:
%python

systems = (
    spark.sql("""
        SELECT DISTINCT name
        FROM edsm.silver.systemsWithCoordinates
        WHERE coords.x BETWEEN -150 AND 150
          AND coords.y BETWEEN -100 AND 100
          AND coords.z BETWEEN -150 AND 150
          AND name IS NOT NULL
        ORDER BY name
        LIMIT 1023
    """)
    .collect()
)

system_list = [row.name for row in systems]

if system_list:
    dbutils.widgets.combobox("Commander System", system_list[0], system_list)
else:
    dbutils.widgets.text("Commander System", "")

In [0]:
%python

states = (
    spark.sql("""
        SELECT DISTINCT state
        FROM edsm.silver.systemsPopulated
        WHERE state IS NOT NULL
        ORDER BY state
    """)
    .collect()
)

state_list = [row.state for row in states]

if state_list:
    dbutils.widgets.combobox("System State", state_list[0], state_list)
else:
    dbutils.widgets.text("System State", "")

In [0]:
WITH target_system AS (
    SELECT coords.x AS tx, coords.y AS ty, coords.z AS tz
    FROM edsm.silver.systemsWithCoordinates
    WHERE name = :`Commander System`
),
filtered_systems AS (
    SELECT *
    FROM edsm.silver.systemsPopulated sp
    CROSS JOIN target_system t
    WHERE sp.state = :`System State`
      AND ABS(sp.coords.x - t.tx) <= 100
      AND ABS(sp.coords.y - t.ty) <= 100
      AND ABS(sp.coords.z - t.tz) <= 100
),
with_distance AS (
    SELECT
        name,
        coords.x,
        coords.y,
        coords.z,
        SQRT(
            POW(coords.x - t.tx, 2) +
            POW(coords.y - t.ty, 2) +
            POW(coords.z - t.tz, 2)
        ) AS distance
    FROM filtered_systems
    CROSS JOIN target_system t
)
SELECT *
FROM with_distance
ORDER BY distance ASC

In [0]:
%python

commander = dbutils.widgets.get("Commander System")
state = dbutils.widgets.get("System State")

query = f"""
WITH target_system AS (
    SELECT coords.x AS tx, coords.y AS ty, coords.z AS tz
    FROM edsm.silver.systemsWithCoordinates
    WHERE name = '{commander}'
),
filtered_systems AS (
    SELECT *
    FROM edsm.silver.systemsPopulated sp
    CROSS JOIN target_system t
    WHERE sp.state = '{state}'
      AND ABS(sp.coords.x - t.tx) <= 100
      AND ABS(sp.coords.y - t.ty) <= 100
      AND ABS(sp.coords.z - t.tz) <= 100
),
with_distance AS (
    SELECT
        name,
        coords.x AS x,
        coords.y AS y,
        coords.z AS z,
        SQRT(
            POW(coords.x - t.tx, 2) +
            POW(coords.y - t.ty, 2) +
            POW(coords.z - t.tz, 2)
        ) AS distance
    FROM filtered_systems
    CROSS JOIN target_system t
)
SELECT *
FROM with_distance
ORDER BY distance ASC
"""

df = spark.sql(query).toPandas()

import plotly.express as px

fig = px.scatter_3d(
    df, x='x', y='y', z='z', color='distance', text='name',
    title='Nearby Systems',
    labels={'x': 'X', 'y': 'Y', 'z': 'Z'}
)

fig.update_traces(marker=dict(size=5), textposition='top center')
fig.update_layout(
    width=800,
    height=600,
    scene=dict(
        xaxis=dict(range=[-400, -250]),
        yaxis=dict(range=[-50, 50]),
        zaxis=dict(range=[-350, -200])
    ),
    scene_camera=dict(eye=dict(x=2.5, y=2.5, z=2.5))
)
fig.show()